# Procesamiento de los datos

In [293]:
import pandas as pd
import numpy as np
from typing import List
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize
from nltk.stem import SnowballStemmer
import matplotlib.pyplot as plt
import os

## Preprocesamiento de texto

In [294]:

PATH_TO_DATA = "data/"
PATH_TO_RESULTS = "results/"

docs_raw_path = f"{PATH_TO_DATA}docs-raw-texts/"
queries_raw_path = f"{PATH_TO_DATA}queries-raw-texts/"
bsii_results_file = f"{PATH_TO_RESULTS}BSII-AND-queries_results.tsv"




In [295]:
def normalize_text(text: str) -> str:
    """
    Normaliza el texto eliminando caracteres no deseados y convirtiendo a minúsculas.

    Args:
        text (str): El texto a normalizar.

    Returns:
        str: El texto normalizado.
    """
    text = text.strip().lower()
    text = re.sub(r"\s+", " ", text)  # Reemplaza múltiples espacios por uno solo
    text = re.sub(r'\[\d+\]', '', text)  # Elimina referencias numéricas
    return text

In [296]:
def tokenize_text(text: str) -> List[str]:
    """
    Tokeniza el texto en una lista de tokens utilizando expresiones regulares.

    Args:
        text (str): El texto a tokenizar.

    Returns:
        List[str]: La lista de tokens.
    """
    text = normalize_text(text)
    pattern = r'''(?x)
        (?:[A-Z]\.)+[A-Z]?                      # abreviaturas: U.S.A, U.S.A.
    | [A-Za-z]+(?:-[A-Za-z]+)*                  # palabras con guiones
    | [A-Za-z]+(?:'[A-Za-z]+)?                  # contracciones: don't, we're
    | \$?\d+(?:\.\d+)?%?                        # números simples
    | \.\.\.                                    # puntos suspensivos
    | [\[\].,;"'?():_`!-]                       # puntuación expandida
    '''
    tokens = nltk.regexp_tokenize(text, pattern)
    return tokens


In [297]:
def remove_stopwords(tokens: List[str]) -> List[str]:
    """
    Elimina las stopwords de una lista de tokens.

    Args:
        tokens (List[str]): La lista de tokens a procesar.

    Returns:
        List[str]: La lista de tokens sin stopwords.
    """
    stop_words = set(stopwords.words("english"))
    return [token for token in tokens if token not in stop_words]

In [298]:
def stem_tokens(tokens: List[str]) -> List[str]:
    """
    Aplica stemming a una lista de tokens.

    Args:
        tokens (List[str]): La lista de tokens a procesar.

    Returns:
        List[str]: La lista de tokens con stemming aplicado.
    """
    stemmer = SnowballStemmer("english")
    return [stemmer.stem(token) for token in tokens]


In [299]:
def preprocess_text(text: str) -> List[str]:
    """
    Preprocesa el texto aplicando normalización, tokenización, eliminación de stopwords y stemming.

    Args:
        text (str): El texto a preprocesar.

    Returns:
        List[str]: La lista de tokens preprocesados.
    """
    tokens = tokenize_text(text)
    tokens = remove_stopwords(tokens)
    tokens = stem_tokens(tokens)
    return tokens

## Carga de datos

In [300]:
from pathlib import Path
import xml.etree.ElementTree as ET

def load_naf_data(dir_naf:str) -> dict:
    """
    Carga los datos en formato NAF desde un directorio dado.
    Para cada documento se concatena el título y el texto en bruto.
    Luego de eso se procesa el texto, se tokeniza y se devuelve.

    Args:
        dir_naf (str): El directorio que contiene los archivos NAF.

    Returns:
        dict: Un diccionario con los IDs de los documentos como claves y listas de tokens como valores.
    """
    dirp = Path(dir_naf)
    out = {}
    for naf_path in sorted(dirp.glob("*.naf")):
        root = ET.parse(naf_path).getroot()
        public_id = root.find("./nafHeader/public").attrib["publicId"].strip()
        title = root.find("./nafHeader/fileDesc").attrib["title"].strip()
        raw_text = root.find("raw").text
        payload = raw_text[9:-3].strip()
        combinado = f"{title} {payload}"
        tokens = preprocess_text(combinado)
        out[public_id] = tokens

    return out

In [301]:
documents = load_naf_data(docs_raw_path)

In [302]:
documents.get("d006", [])

['eugenio',
 'beltrami',
 'non-euclidian',
 'geometri',
 'eltrami',
 'non-euclidian',
 'geometri',
 '.',
 'eugenio',
 'beltrami',
 '(',
 '1835',
 '-',
 '1900',
 ')',
 '.',
 'novemb',
 '16',
 ',',
 '1835',
 ',',
 'italian',
 'mathematician',
 'eugenio',
 'beltrami',
 'born',
 '.',
 'notabl',
 'work',
 'concern',
 'differenti',
 'geometri',
 'mathemat',
 'physic',
 '.',
 'work',
 'note',
 'especi',
 'clariti',
 'exposit',
 '.',
 'first',
 'prove',
 'consist',
 'non-euclidean',
 'geometri',
 'model',
 'surfac',
 'constant',
 'curvatur',
 ',',
 'pseudospher',
 '.',
 'eugenio',
 'beltrami',
 'born',
 'cremona',
 'lombardi',
 ',',
 'part',
 'austrian',
 'empir',
 ',',
 'part',
 'itali',
 '.',
 'son',
 'artist',
 'paint',
 'miniatur',
 ',',
 'young',
 'eugenio',
 'certain',
 'inherit',
 'artist',
 'talent',
 'famili',
 ',',
 'case',
 'addit',
 'mathemat',
 'talent',
 'would',
 'acquir',
 ',',
 'musicrath',
 'paint',
 'becam',
 'import',
 'life',
 '.',
 'began',
 'studi',
 'mathemat',
 'univer

## Índice invertido (Inverted Index) (BSII) 

In [303]:
def build_inverted_index(documents: dict) -> dict:
    """
    Construye un índice invertido a partir de los documentos tokenizados.

    Args:
        documents (dict): Un diccionario con los IDs de los documentos como claves y listas de tokens como valores.

    Returns:
        dict: Un índice invertido donde cada término tiene 'df' (document frequency) y 'postings' (lista ordenada de doc_ids).
    """
    inverted_index = {}
    
    for doc_id, tokens in documents.items():
        unique_tokens = set(tokens)
        for token in unique_tokens:
            if token not in inverted_index:
                inverted_index[token] = []
            inverted_index[token].append(doc_id)
    
    # Convertir a formato con df y postings ordenados
    for token in inverted_index:
        postings = sorted(inverted_index[token])  # Ordenar posting list
        inverted_index[token] = {
            'docfreq': len(postings),                  # Document frequency
            'postings': postings                  # Posting list ordenada
        }
    
    return inverted_index

In [304]:
inverted_index = build_inverted_index(documents)

In [305]:
inverted_index.get("exam", {})

{'docfreq': 5, 'postings': ['d009', 'd189', 'd194', 'd283', 'd325']}

In [306]:
inverted_index.get("beltrami", {})

{'docfreq': 2, 'postings': ['d006', 'd262']}

### Funciones de búsqueda booleana

Manning et al., Capítulo 1.3: "Processing Boolean queries"


In [307]:
def merge_and(list1: List[str], list2: List[str]) -> List[str]:
    """
    Mezcla dos listas ORDENADAS de resultados de búsqueda utilizando la operación AND.

    Se emplean dos punteros para recorrer ambas listas de manera eficiente.

    Args:
        list1 (List[str]): La primera lista de resultados.
        list2 (List[str]): La segunda lista de resultados.

    Returns:
        List[str]: Una lista ordenada de resultados que están en ambas listas.
    """
    i, j = 0, 0
    result = []
    while i < len(list1) and j < len(list2):
        if list1[i] == list2[j]:
            result.append(list1[i])  # Documento en ambas listas
            i += 1
            j += 1
        elif list1[i] < list2[j]:
            i += 1  # Avanza en list1
        else:
            j += 1  # Avanza en list2

    return result

In [308]:
def merge_not(list1: List[str], list2: List[str]) -> List[str]:
    """
    Mezcla dos listas ORDENADAS de resultados de búsqueda utilizando la operación NOT.
    Implementa una variación del algoritmo del recorrido con dos punteros.

    Args:
        list1 (List[str]): La primera lista de resultados.
        list2 (List[str]): La segunda lista de resultados.

    Returns:
        List[str]: Una lista ordenada de los documentos que están en la primera lista pero no en la segunda.
    """
    i, j = 0, 0
    result = []
    
    while i < len(list1):
        if j < len(list2) and list1[i] == list2[j]:
            # Documento está en ambas - excluir
            i += 1
            j += 1
        elif j < len(list2) and list1[i] > list2[j]:
            # Avanzar j hasta alcanzar o pasar list1[i]
            j += 1
        else:
            # list1[i] no está en list2 (o j se agotó) - incluir
            result.append(list1[i])
            i += 1
    
    return result

In [309]:
def boolean_search(inverted_index: dict, query_terms: List[str], operation: str) -> List[str]:
    """
    Ejecuta consultas booleanas usando el algoritmo de mezcla.

    La función soporta operaciones 'AND' y 'NOT'. Sin embargo, es importante tener en cuenta que:
    - En una operación 'AND', si algún término no se encuentra en el índice invertido, el resultado será vacío.
    - En una operación 'NOT', si el término base (el primer término) no se encuentra, el resultado será vacío.

    Args:
        inverted_index (dict): El índice invertido.
        query_terms (List[str]): Los términos de búsqueda.
        operation (str): La operación booleana ('AND' o 'NOT').

    Returns:
        List[str]: Los documentos que coinciden con la consulta.
    """
    if operation not in ['AND', 'NOT']:
        raise ValueError("Operation must be 'AND' or 'NOT'")
    
    if not query_terms:
        return []
    
    if operation == 'AND':
        posting_lists = []
        for term in query_terms:
            if term in inverted_index:
                posting_lists.append(inverted_index[term]['postings'])
            else:
                return []
        
        if not posting_lists:
            return []
        result = posting_lists[0]
        for posting_list in posting_lists[1:]:
            result = merge_and(result, posting_list)
            if not result:
                return []
        return result
    
    elif operation == 'NOT':
        
        if query_terms[0] not in inverted_index:
            return []
        
        result = inverted_index[query_terms[0]]['postings'].copy()
        
        for term in query_terms[1:]:
            if term in inverted_index:
                result = merge_not(result, inverted_index[term]['postings'])
                if not result: 
                    return []
        
        return result

In [310]:
def process_all_and_queries(inverted_index: dict, queries_folder: str, output_file: str):
    """
    Procesa todas las consultas AND desde archivos NAF y escribe resultados.


    """
    if not os.path.exists(os.path.dirname(output_file)):
        os.makedirs(os.path.dirname(output_file))
    with open(output_file, 'w') as f:

        query_files = sorted([file for file in os.listdir(queries_folder) if file.endswith('.naf')])

        for query_file in query_files:
            # Parsear XML NAF
            tree = ET.parse(os.path.join(queries_folder, query_file))
            raw_content = tree.find('.//raw').text.strip()

            public_id = tree.find('.//public').get('publicId') 
            
            query_tokens = preprocess_text(raw_content)

            results = boolean_search(inverted_index, query_tokens, 'AND')
            if results:
                f.write(f"{public_id} {','.join(results)}\n")
            else:
                f.write(f"{public_id}\n")

In [311]:
inverted_index.get("instrument", {})

{'docfreq': 35,
 'postings': ['d016',
  'd021',
  'd024',
  'd028',
  'd032',
  'd038',
  'd060',
  'd074',
  'd082',
  'd085',
  'd094',
  'd099',
  'd100',
  'd116',
  'd123',
  'd153',
  'd170',
  'd172',
  'd179',
  'd186',
  'd212',
  'd229',
  'd234',
  'd243',
  'd254',
  'd255',
  'd265',
  'd273',
  'd275',
  'd281',
  'd284',
  'd299',
  'd315',
  'd317',
  'd329']}

In [312]:
process_all_and_queries(inverted_index, queries_raw_path, bsii_results_file)